In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import mixed_precision

BATCH_SIZE = 512
NUM_CLASSES = 62
EPOCHS = 100

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
tf.config.optimizer.set_jit(True)

def fix_orientation(images):
    images = np.rot90(images, k=3, axes=(1, 2))
    images = np.flip(images, axis=2)
    return images

df_train = pd.read_csv("./emnist-byclass-train.csv", header=None)
df_test = pd.read_csv("./emnist-byclass-test.csv", header=None)

X = df_train.drop(columns=[0]).to_numpy().reshape(-1, 28, 28, 1)
y = df_train[0]

X_test = df_test.drop(columns=[0]).to_numpy().reshape(-1, 28, 28, 1)
y_test = df_test[0]

X = fix_orientation(X).astype(np.float32) / 255.0
X_test = fix_orientation(X_test).astype(np.float32) / 255.0

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

y_train_oh = tf.one_hot(y_train, NUM_CLASSES)
y_val_oh = tf.one_hot(y_val, NUM_CLASSES)
y_test_oh = tf.one_hot(y_test, NUM_CLASSES)

data_augmentation = tf.keras.Sequential([
    layers.RandomRotation(0.05),
    layers.RandomTranslation(0.05, 0.05),
    layers.RandomZoom(0.05, 0.05),
    layers.RandomContrast(0.05),
    layers.GaussianNoise(0.05)
])

def augment(images, labels):
    images = data_augmentation(images)
    return images, labels

train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train_oh))
    .shuffle(10000)
    .batch(BATCH_SIZE)
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

val_ds = (
    tf.data.Dataset.from_tensor_slices((X_val, y_val_oh))
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test_oh))
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

def residual_block(x, filters, stride=1):
    shortcut = x
    x = layers.Conv2D(filters, (3, 3), strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, (3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), strides=stride, padding='same', use_bias=False)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
        
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

inputs = layers.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, (3, 3), padding='same', use_bias=False)(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = residual_block(x, 32, stride=1)

x = residual_block(x, 64, stride=2)
x = residual_block(x, 64, stride=1)

x = residual_block(x, 128, stride=2)
x = residual_block(x, 128, stride=1)

x = residual_block(x, 256, stride=2)
x = residual_block(x, 256, stride=1)

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(128, use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Dropout(0.6)(x)

outputs = layers.Dense(NUM_CLASSES, activation='softmax', dtype='float32')(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.CosineDecay(
            initial_learning_rate=5e-4,
            decay_steps=len(train_ds) * EPOCHS,
            alpha=0.001
        )
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy']
)

model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=17,
        restore_best_weights=True
    )],
    verbose=2
)

pred_probs = model.predict(test_ds)
pred_labels = tf.argmax(pred_probs, axis=1)
accuracy = accuracy_score(y_test, pred_labels)
print("Test accuracy:", accuracy)


2025-08-15 22:10:54.457328: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-15 22:10:54.457358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-15 22:10:54.458336: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-15 22:10:54.463540: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-15 22:10:55.012896: W tensorflow/compiler/tf2

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050, compute capability 8.6


2025-08-15 22:10:55.998100: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-15 22:10:56.030020: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-15 22:10:56.032442: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/100


2025-08-15 22:11:25.963549: I external/local_xla/xla/service/service.cc:168] XLA service 0x10bba2280 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-15 22:11:25.963578: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050, Compute Capability 8.6
2025-08-15 22:11:25.974196: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-15 22:11:26.087228: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1755267091.067693  179800 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-08-15 22:11:32.328973: I external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:326] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_328', 212 bytes spill 

1227/1227 - 99s - loss: 0.9901 - accuracy: 0.8259 - val_loss: 0.8134 - val_accuracy: 0.8495 - 99s/epoch - 80ms/step
Epoch 2/100


2025-08-15 22:14:06.316536: I external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:326] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_463', 76 bytes spill stores, 76 bytes spill loads

2025-08-15 22:14:06.791021: I external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:326] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_328', 580 bytes spill stores, 564 bytes spill loads

2025-08-15 22:14:06.911684: I external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:326] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_463', 236 bytes spill stores, 236 bytes spill loads

2025-08-15 22:14:07.106094: I external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:326] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_463', 152 bytes spill stores, 152 bytes spill loads

2025-08-15 22:14:07.181388: I external/local_xla/xla/stream_executor/gpu/asm_compi

1227/1227 - 88s - loss: 0.7632 - accuracy: 0.8863 - val_loss: 0.7158 - val_accuracy: 0.8748 - 88s/epoch - 72ms/step
Epoch 3/100
1227/1227 - 62s - loss: 0.7338 - accuracy: 0.8930 - val_loss: 0.6720 - val_accuracy: 0.8864 - 62s/epoch - 50ms/step
Epoch 4/100
1227/1227 - 62s - loss: 0.7193 - accuracy: 0.8964 - val_loss: 0.6781 - val_accuracy: 0.8831 - 62s/epoch - 50ms/step
Epoch 5/100
1227/1227 - 62s - loss: 0.7095 - accuracy: 0.8983 - val_loss: 0.6543 - val_accuracy: 0.8928 - 62s/epoch - 50ms/step
Epoch 6/100
1227/1227 - 62s - loss: 0.7009 - accuracy: 0.9004 - val_loss: 0.6555 - val_accuracy: 0.8897 - 62s/epoch - 51ms/step
Epoch 7/100
1227/1227 - 62s - loss: 0.6938 - accuracy: 0.9022 - val_loss: 0.6566 - val_accuracy: 0.8897 - 62s/epoch - 50ms/step
Epoch 8/100
1227/1227 - 62s - loss: 0.6889 - accuracy: 0.9030 - val_loss: 0.6506 - val_accuracy: 0.8888 - 62s/epoch - 50ms/step
Epoch 9/100
1227/1227 - 62s - loss: 0.6848 - accuracy: 0.9040 - val_loss: 0.6382 - val_accuracy: 0.8967 - 62s/epoch 

2025-08-15 22:52:39.513414: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 364788928 exceeds 10% of free system memory.


228/228 [==============================] - 5s 17ms/step
Test accuracy: 0.9077396559579791


In [4]:
model.save('HR', save_format='tf')

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 1, 32), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa43b0dba90>, 140344908904000), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 1, 32), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa43b0dba90>, 140344908904000), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 32, 32), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa43d541950>, 140343354369936), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 32, 32), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa43d541950>, 140343354369936), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 32, 32), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa43d58ef90>, 140343354375376), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 256, 256), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa48dff63d0>, 140343354800400), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(256, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa48dffefd0>, 140343354383760), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(256, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa48dffefd0>, 140343354383760), {}).


INFO:tensorflow:Assets written to: HR/assets


INFO:tensorflow:Assets written to: HR/assets


In [5]:
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model HR tfjs_model_bymerge

2025-08-15 23:06:49.575037: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-15 23:06:49.575074: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-15 23:06:49.576095: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-15 23:06:50.127358: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
weight StatefulPartitionedCall/model/dense/MatMul/Cast with shape (256, 128) and dtype float16 was auto converted to the type float32
weight StatefulPartitionedCall/model/conv2d/Conv2D_weights with shape (3, 3, 1, 32) and dtype float16 was a

In [ ]:
import matplotlib.pyplot as plt

for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(X_test[i].squeeze(), cmap='gray')
    plt.title(f"T: {y_test.iloc[i]}\nP: {pred_labels[i]}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
#letters Test accuracy: 0.9436486486486486